In [1]:
import re
import numpy as np

In [2]:
rotation_map_2 = {
    0: 1,
    1: 3,
    2: 0,
    3: 2,    
}

rotation_map_3 = {
    0: 2,
    1: 5,
    2: 8,
    3: 1,
    4: 4,
    5: 7,
    6: 0,
    7: 3,
    8: 6,
}

reflection_map_2 = {
    0: 2,
    1: 3,
    2: 0,
    3: 1,
}

reflection_map_3 = {
    0: 6,
    1: 7,
    2: 8,
    3: 3,
    4: 4,
    5: 5,
    6: 0,
    7: 1,
    8: 2,
}

def grid_op (grid, op_map):
    output = list(grid)
    for old_pos, new_pos in op_map.items():
        output[new_pos] = grid[old_pos]
    return output

rot2 = lambda x: grid_op(x, rotation_map_2)
refl2 = lambda x: grid_op(x, reflection_map_2)
rot3 = lambda x: grid_op(x, rotation_map_3)
refl3 = lambda x: grid_op(x, reflection_map_3)

In [3]:
test_grid = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']
assert rot3(rot3(rot3(test_grid))) == ['c', 'f', 'i', 'b', 'e', 'h', 'a', 'd', 'g']

In [4]:
def all_3_grids(grid):
    return [
        grid,
        rot3(grid),
        rot3(rot3(grid)),
        rot3(rot3(rot3(grid))),
        refl3(grid),
        rot3(refl3(grid)),
        rot3(rot3(refl3(grid))),
        rot3(rot3(rot3(refl3(grid)))),        
    ]

def all_2_grids(grid):
    return [
        grid,
        rot2(grid),
        rot2(rot2(grid)),
        rot2(rot2(rot2(grid))),
        refl2(grid),
        rot2(refl2(grid)),
        rot2(rot2(refl2(grid))),
        rot2(rot2(rot2(refl2(grid)))),        
    ]

In [5]:
all_3_grids(test_grid)

[['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'],
 ['g', 'd', 'a', 'h', 'e', 'b', 'i', 'f', 'c'],
 ['i', 'h', 'g', 'f', 'e', 'd', 'c', 'b', 'a'],
 ['c', 'f', 'i', 'b', 'e', 'h', 'a', 'd', 'g'],
 ['g', 'h', 'i', 'd', 'e', 'f', 'a', 'b', 'c'],
 ['a', 'd', 'g', 'b', 'e', 'h', 'c', 'f', 'i'],
 ['c', 'b', 'a', 'f', 'e', 'd', 'i', 'h', 'g'],
 ['i', 'f', 'c', 'h', 'e', 'b', 'g', 'd', 'a']]

In [6]:
def binary_grid_to_num(grid):
    """Expecting lists of []'0', '1']"""
    if len(grid) == 9:
        grids = all_3_grids(grid)
    else:
        grids = all_2_grids(grid)
    return min(int(''.join(g), 2) for g in grids)

In [7]:
assert binary_grid_to_num(['1', '1', '0', '1', '1', '1', '1', '1', '1']) == 255
assert binary_grid_to_num(['0', '1', '1', '1', '1', '1', '1', '1', '1']) == 255
assert binary_grid_to_num(['1', '0', '0', '1']) == 6

```
##/#. => .../.#./..#
##/## => #.#/.##/.##
.../.../... => ##../.#../##../#..#
#../.../... => ..#./##.#/#.##/....
```

In [8]:
fractal_re = re.compile('^([/#.]+) => ([/#.]+)$')

In [9]:
m = fractal_re.match('.../.../... => ##../.#../##../#..#')
print(m[1], m[2])

.../.../... ##../.#../##../#..#


In [10]:
def grid_string_to_num(grid_str):
    ret = grid_str
    for k, v in {'#': '1', '.': '0', '/': ''}.items():
        ret = ret.replace(k, v)
    return binary_grid_to_num(list(ret))

assert grid_string_to_num('#./.#') == 6

In [11]:
def input_to_mappings(lines):
    fractal_map = dict()
    for l in lines:
        m = fractal_re.match(l)
        if not m:
            continue
        size = 2 if len(m[1]) == 5 else 3
        key = (size, grid_string_to_num(m[1]))
        val = [list(l) for l in m[2].split('/')]
        fractal_map[key] = val
    return fractal_map

In [12]:
test_lines = [
    '../.# => ##./#../...',
    '.#./..#/### => #..#/..../..../#..#'
]
input_to_mappings(test_lines)

{(2, 1): [['#', '#', '.'], ['#', '.', '.'], ['.', '.', '.']],
 (3, 107): [['#', '.', '.', '#'],
  ['.', '.', '.', '.'],
  ['.', '.', '.', '.'],
  ['#', '.', '.', '#']]}

In [13]:
def split_grid(grid):
    size = 2 if len(grid) % 2 == 0 else 3
    arr = np.array(grid)
    output = list()
    for x in range(0, len(arr)- 1, size):
        for y in range(0, len(arr)- 1, size):
            output.append(list(arr[x:x+size,y:y+size].flatten()))
    return output

In [14]:
assert split_grid([['a', 'b'], ['c', 'd']]) == [['a', 'b', 'c', 'd']]
assert split_grid([
    ['a', 'b', 'c', 'd'],
    ['e', 'f', 'g', 'h'],
    ['i', 'j', 'k', 'l'],
    ['m', 'n', 'o', 'p']]
) == [
    ['a', 'b', 'e', 'f'],
    ['c', 'd', 'g', 'h'],
    ['i', 'j', 'm', 'n'],
    ['k', 'l', 'o', 'p']]
assert split_grid([
    ['a', 'b', 'c'],
    ['d', 'e', 'f'],
    ['g', 'h', 'i'],
]) == [['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']]
assert split_grid([
    ['a', 'b', 'c', 'd', 'e', 'f'],
    ['g', 'h', 'i', 'j', 'k', 'l'],
    ['m', 'n', 'o', 'p', 'q', 'r'],
    ['s', 't', 'u', 'v', 'w', 'x'],
    ['y', 'z', '0', '1', '2', '3'],
    ['4', '5', '6', '7', '8', '9'],
]) == [
    ['a', 'b', 'g', 'h'],
    ['c', 'd', 'i', 'j'],
    ['e', 'f', 'k', 'l'],
    ['m', 'n', 's', 't'],
    ['o', 'p', 'u', 'v'],
    ['q', 'r', 'w', 'x'],
    ['y', 'z', '4', '5'],
    ['0', '1', '6', '7'],
    ['2', '3', '8', '9']
]

In [15]:
l = split_grid(np.arange(81).reshape((9, 9)))
assert l[0] == [0, 1, 2, 9, 10, 11, 18, 19, 20]
assert l[-1] == [60, 61, 62, 69, 70, 71, 78, 79, 80]

In [16]:
def join_grids(grids, col_size):
    size = len(grids[0]) 
    cols = size * col_size
    rows = len(grids) // col_size * size
    arr = np.empty((rows, cols), dtype='object')
    i = 0
    for x in range(0, len(arr), size):
        for y in range(0, len(arr), size):
            arr[x:x+size,y:y+size] = grids[i]
            i += 1
    return arr.tolist()

assert join_grids([
    [['0', '1'], ['2', '3']],
    [['4', '5'], ['6', '7']],
    [['8', '9'], ['10', '11']],
    [['12', '13'], ['14', '15']],
], 2) == [
    ['0', '1', '4', '5'],
    ['2', '3', '6', '7'],
    ['8', '9', '12', '13'],
    ['10', '11', '14', '15'], 
]

l = join_grids([
    [['#', '#', '.'], ['.', '.', '.'], ['.', '.', '#']],
    [['.', '#', '.'], ['.', '#', '.'], ['#', '#', '.']],
    [['.', '#', '.'], ['.', '#', '.'], ['#', '#', '.']],
    [['#', '#', '.'], ['.', '.', '.'], ['.', '.', '#']],
    [['.', '#', '.'], ['.', '#', '.'], ['#', '#', '.']],
    [['#', '.', '.'], ['.', '.', '#'], ['#', '#', '.']],
    [['.', '.', '.'], ['.', '#', '#'], ['.', '#', '#']],
    [['#', '#', '.'], ['.', '.', '.'], ['.', '.', '#']],
    [['#', '#', '.'], ['.', '.', '.'], ['.', '.', '#']]
], 3)
assert l[0] == ['#', '#', '.', '.', '#', '.', '.', '#', '.']
assert l[-1] == ['.', '#', '#', '.', '.', '#', '.', '.', '#']

In [17]:
def play(initial_grid, fractal_map):
    mini_grids = list()
    for s in split_grid(initial_grid):
        size = 2 if len(s) == 4 else 3 
        col_len = len(initial_grid) // size
        key = (size, grid_string_to_num(''.join(s)))
        mini_grids.append(fractal_map[key])
    grid = join_grids(mini_grids, col_len)
    return grid

Initial grid:
```
.#.
..#
###
```

In [18]:
initial_grid = [['.', '#', '.'], ['.', '.', '#'], ['#', '#', '#']]
g = play(initial_grid, input_to_mappings(test_lines))
print(g)
play(g, input_to_mappings(test_lines))

[['#', '.', '.', '#'], ['.', '.', '.', '.'], ['.', '.', '.', '.'], ['#', '.', '.', '#']]


[['#', '#', '.', '#', '#', '.'],
 ['#', '.', '.', '#', '.', '.'],
 ['.', '.', '.', '.', '.', '.'],
 ['#', '#', '.', '#', '#', '.'],
 ['#', '.', '.', '#', '.', '.'],
 ['.', '.', '.', '.', '.', '.']]

In [19]:
with open('fractal_art.txt') as fh:
    lines = fh.readlines()
lines = [l.strip() for l in lines]
fractal_mapping = input_to_mappings(lines)

In [20]:
grid = [['.', '#', '.'], ['.', '.', '#'], ['#', '#', '#']]
for i in range(5):
    grid = play(grid, fractal_mapping)

In [21]:
c = 0
for i in grid:
    c += sum(1 for j in i if j == '#')
c

123

In [22]:
grid = [['.', '#', '.'], ['.', '.', '#'], ['#', '#', '#']]
for i in range(18):
    grid = play(grid, fractal_mapping)

In [23]:
c = 0
for i in grid:
    c += sum(1 for j in i if j == '#')
c

1984683